In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

import matplotlib.pyplot as plt

In [2]:
batch_size = 128
epochs = 10
seed = 1
log_interval=10

In [3]:
device = torch.device("cpu")

kwargs = {} #{'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

In [4]:
class CVAE(nn.Module):
    def __init__(self):
        super(CVAE, self).__init__()
        
        num_latent = 20
        #So here we will first define layers for encoder network
        self.conv = nn.Sequential(nn.Conv2d(1, 3, 3, padding=1),
                                     nn.MaxPool2d(2, 2),
                                     nn.BatchNorm2d(3),
                                     nn.Conv2d(3, 16, 3, padding=1),
                                     nn.MaxPool2d(2, 2),
                                     nn.BatchNorm2d(16),
                                     nn.Conv2d(16, 16, 3, padding=1))
        
        #These two layers are for getting logvar and mean
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.mean = nn.Linear(128, num_latent)
        self.var = nn.Linear(128, num_latent)
        
        #######The decoder part
        #This is the first layer for the decoder part
        self.expand = nn.Linear(num_latent, 128)
        self.fc3 = nn.Linear(128, 256)
        self.fc4 = nn.Linear(256, 784)
        self.deconv = nn.Sequential(nn.ConvTranspose2d(16, 16, 3, padding=1),
                                     nn.BatchNorm2d(16),
                                     nn.ConvTranspose2d(16, 3, 8),
                                     nn.BatchNorm2d(3),
                                     nn.ConvTranspose2d(3, 1, 15))
        
    def encoder(self, x):
        #here we will be returning the logvar(log variance) and mean of our network
        x = self.conv(x)
        x = x.view([-1, 784])
        x = F.dropout2d(self.fc1(x), 0.5)
        x = self.fc2(x)
        
        mean = self.mean(x)
        logvar = self.var(x)
        return mean, logvar
    
    def decoder(self, z):
        #here z is the latent variable state
        z = self.expand(z)
        z = F.dropout2d(self.fc3(z), 0.5)
        z = self.fc4(z)
        z = z.view([-1, 16, 7, 7])
        
        out = self.deconv(z)
        out = F.sigmoid(out)
        return out
    
    def get_hidden(self, mean, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)   #So as to get std
            noise = torch.randn_like(mean)   #So as to get the noise of standard distribution
            return noise.mul(std).add_(mean)
        else:
            return mean
    
    def forward(self, x):
        mean, logvar = self.encoder(x)
        z = self.get_hidden(mean, logvar)
        out = self.decoder(z)
        return out, mean, logvar

model = CVAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [12]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [15]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [17]:
if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decoder(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'results/cvae_sample_' + str(epoch) + '.png')

/anaconda3/envs/torchenv/lib/python3.5/site-packages/torch/nn/functional.py:1101: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/anaconda3/envs/torchenv/lib/python3.5/site-packages/torch/nn/functional.py:1690: UserWarning: Using a target size (torch.Size([128, 784])) that is different to the input size (torch.Size([128, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Train Epoch: 1 [0/60000 (0%)]	Loss: 149.853729
Train Epoch: 1 [1280/60000 (2%)]	Loss: 155.198502
Train Epoch: 1 [2560/60000 (4%)]	Loss: 153.690384
Train Epoch: 1 [3840/60000 (6%)]	Loss: 149.594986
Train Epoch: 1 [5120/60000 (9%)]	Loss: 155.983917
Train Epoch: 1 [6400/60000 (11%)]	Loss: 154.539734
Train Epoch: 1 [7680/60000 (13%)]	Loss: 157.018814
Train Epoch: 1 [8960/60000 (15%)]	Loss: 151.043869
Train Epoch: 1 [10240/60000 (17%)]	Loss: 153.578232
Train Epoch: 1 [11520/60000 (19%)]	Loss: 156.561264
Train Epoch: 1 [12800/60000 (21%)]	Loss: 158.006378
Train Epoch: 1 [14080/60000 (23%)]	Loss: 152.053909
Train Epoch: 1 [15360/60000 (26%)]	Loss: 159.709885
Train Epoch: 1 [16640/60000 (28%)]	Loss: 150.218765
Train Epoch: 1 [17920/60000 (30%)]	Loss: 152.778381
Train Epoch: 1 [19200/60000 (32%)]	Loss: 148.353958
Train Epoch: 1 [20480/60000 (34%)]	Loss: 148.119858
Train Epoch: 1 [21760/60000 (36%)]	Loss: 155.311432
Train Epoch: 1 [23040/60000 (38%)]	Loss: 151.528717
Train Epoch: 1 [24320/60000 

/anaconda3/envs/torchenv/lib/python3.5/site-packages/torch/nn/functional.py:1690: UserWarning: Using a target size (torch.Size([96, 784])) that is different to the input size (torch.Size([96, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


====> Epoch: 1 Average loss: 152.9104


/anaconda3/envs/torchenv/lib/python3.5/site-packages/torch/nn/functional.py:1690: UserWarning: Using a target size (torch.Size([16, 784])) that is different to the input size (torch.Size([16, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


====> Test set loss: 147.6620
Train Epoch: 2 [0/60000 (0%)]	Loss: 152.502411
Train Epoch: 2 [1280/60000 (2%)]	Loss: 150.533997
Train Epoch: 2 [2560/60000 (4%)]	Loss: 151.485214
Train Epoch: 2 [3840/60000 (6%)]	Loss: 146.579575
Train Epoch: 2 [5120/60000 (9%)]	Loss: 147.499924
Train Epoch: 2 [6400/60000 (11%)]	Loss: 156.513397
Train Epoch: 2 [7680/60000 (13%)]	Loss: 154.496445
Train Epoch: 2 [8960/60000 (15%)]	Loss: 155.963440
Train Epoch: 2 [10240/60000 (17%)]	Loss: 146.618256
Train Epoch: 2 [11520/60000 (19%)]	Loss: 151.248596
Train Epoch: 2 [12800/60000 (21%)]	Loss: 152.410461
Train Epoch: 2 [14080/60000 (23%)]	Loss: 160.667007
Train Epoch: 2 [15360/60000 (26%)]	Loss: 153.902176
Train Epoch: 2 [16640/60000 (28%)]	Loss: 154.143997
Train Epoch: 2 [17920/60000 (30%)]	Loss: 151.890793
Train Epoch: 2 [19200/60000 (32%)]	Loss: 155.850006
Train Epoch: 2 [20480/60000 (34%)]	Loss: 151.271774
Train Epoch: 2 [21760/60000 (36%)]	Loss: 152.922516
Train Epoch: 2 [23040/60000 (38%)]	Loss: 152.09129

Train Epoch: 5 [17920/60000 (30%)]	Loss: 152.528641
Train Epoch: 5 [19200/60000 (32%)]	Loss: 147.597015
Train Epoch: 5 [20480/60000 (34%)]	Loss: 144.497086
Train Epoch: 5 [21760/60000 (36%)]	Loss: 151.695206
Train Epoch: 5 [23040/60000 (38%)]	Loss: 150.676926
Train Epoch: 5 [24320/60000 (41%)]	Loss: 152.684875
Train Epoch: 5 [25600/60000 (43%)]	Loss: 147.417679
Train Epoch: 5 [26880/60000 (45%)]	Loss: 154.165710
Train Epoch: 5 [28160/60000 (47%)]	Loss: 150.173447
Train Epoch: 5 [29440/60000 (49%)]	Loss: 151.642380
Train Epoch: 5 [30720/60000 (51%)]	Loss: 152.528656
Train Epoch: 5 [32000/60000 (53%)]	Loss: 151.717422
Train Epoch: 5 [33280/60000 (55%)]	Loss: 147.728973
Train Epoch: 5 [34560/60000 (58%)]	Loss: 146.251328
Train Epoch: 5 [35840/60000 (60%)]	Loss: 147.767044
Train Epoch: 5 [37120/60000 (62%)]	Loss: 150.789642
Train Epoch: 5 [38400/60000 (64%)]	Loss: 150.149536
Train Epoch: 5 [39680/60000 (66%)]	Loss: 159.019791
Train Epoch: 5 [40960/60000 (68%)]	Loss: 146.676422
Train Epoch:

Train Epoch: 8 [35840/60000 (60%)]	Loss: 154.762817
Train Epoch: 8 [37120/60000 (62%)]	Loss: 150.788940
Train Epoch: 8 [38400/60000 (64%)]	Loss: 152.601425
Train Epoch: 8 [39680/60000 (66%)]	Loss: 151.461578
Train Epoch: 8 [40960/60000 (68%)]	Loss: 152.254166
Train Epoch: 8 [42240/60000 (70%)]	Loss: 146.335876
Train Epoch: 8 [43520/60000 (72%)]	Loss: 147.338516
Train Epoch: 8 [44800/60000 (75%)]	Loss: 148.305511
Train Epoch: 8 [46080/60000 (77%)]	Loss: 145.333969
Train Epoch: 8 [47360/60000 (79%)]	Loss: 143.239838
Train Epoch: 8 [48640/60000 (81%)]	Loss: 152.244141
Train Epoch: 8 [49920/60000 (83%)]	Loss: 149.436676
Train Epoch: 8 [51200/60000 (85%)]	Loss: 147.887085
Train Epoch: 8 [52480/60000 (87%)]	Loss: 142.924088
Train Epoch: 8 [53760/60000 (90%)]	Loss: 147.785278
Train Epoch: 8 [55040/60000 (92%)]	Loss: 144.233185
Train Epoch: 8 [56320/60000 (94%)]	Loss: 154.776154
Train Epoch: 8 [57600/60000 (96%)]	Loss: 152.460754
Train Epoch: 8 [58880/60000 (98%)]	Loss: 148.706467
====> Epoch:

In [20]:
from sklearn.manifold import TSNE

#Here I have just created one more trainloader which can help me out in the displaying of the images
viz_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),batch_size=1000, shuffle=True, **kwargs)
dataiter = iter(viz_loader)
images, labels = dataiter.next()
z = model.encoder(images)

In [21]:
z_embedded = TSNE(n_components=2).fit_transform(z.detach().numpy())

AttributeError: 'tuple' object has no attribute 'detach'

In [ ]:
for i in range(1000):
    if(labels[i].numpy() == 0):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='black')
    elif(labels[i].numpy() == 1):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='red')
    elif(labels[i].numpy() == 2):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='green')
    elif(labels[i].numpy() == 3):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='pink')
    elif(labels[i].numpy() == 4):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='gray')
    elif(labels[i].numpy() == 5):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='purple')
    elif(labels[i].numpy() == 6):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='maroon')
    elif(labels[i].numpy() == 7):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='yellow')
    elif(labels[i].numpy() == 8):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='brown')
    elif(labels[i].numpy() == 9):
        plt.scatter(z_embedded[i, 0], z_embedded[i, 1], c='blue')
        
    plt.plot()